47316 Advanced Computational Tools for Energy Materials

# Volmer-Heyrovsky reaction

In this exercise, we want to calculate the limiting potential for the Volmer-Heyrovsky mechanism for proton reduction to H2. We consider the reaction mechanism consisting of the Volmer step of proton adsorption on an active site, $\ast$, coupled with electron transfer

$$ H^{+} + e^{-} + \ast \rightarrow H^{\ast} $$

followed by the Heyrovsky step

$$ H^{\ast} + e^{-} + H^{\ast} \rightarrow \ast + H_{2} $$

The first step in analyzing the Volmer-Heyrovsky mechanism is to calculate the reaction free energy at 0 V versus the reversible hydrogen electrode (RHE).

## Computational hydrogen electrode

At 0 V measured versus the RHE, the reaction:

$$ H^{+} + e^{-} \leftrightarrow \frac{1}{2} H_{2} $$

is in equilibrium when the H$_2$ partial pressure is 1 bar. Consequently at 0 V vs RHE, the reaction free energy of (1) equals the reaction free energy of the chemical reaction:

$$ \frac{1}{2} H_{2} + \ast \rightarrow H^{\ast} $$

and the reaction free energy of (2) equals the reaction free energy of the chemical reaction:

$$ \frac{1}{2} H_{2} + H^{\ast} \rightarrow \ast + H_{2} $$

## DFT Simulations

The required DFT simulations for this exercise are listed below. It is advantageous to work on these simultaneously. In order to compare DFT total energies, calculators with identical cutoffs should be used for molecules and slabs to allow faster convergence of reaction energies.

1. Calculate the total energy of an H$_2$ molecule with DFT.

2. Use the `ase.vibrations` and `ase.thermochemistry` modules to calculate the vibrational frequency, zero point energy, enthalpy and entropy of the H$_2$ molecule at a temperature of 298.15 K within the ideal gas approximation (called `IdealGasThermo` in ASE).

3. Calculate the total energy of the pristine slab, $\ast$, and the slab with an H adsorbate, $H^{\ast}$. You may reuse structures from the first homework problem.

4. Use the `ase.vibrations` and `ase.thermochemistry` modules to calculate the vibrational frequencies, zero point energy, internal energy and entropy at temperature of 298.15 K for the slab with an H$^{\ast}$ adsorbate within the Harmonic approximation (called `HarmonicThermo` in ASE). Assume the internal energy and enthalpy are equal ($U=H$), which implies that the Helmholtz free energy and Gibbs free energy are equal, too ($F=G$). We neglect zero point energy, internal energy, and entropy related to the metal atoms, i.e. we do not perform a vibrational analysis for the pristine slab.

Fill out the table below based on your simulations:

| **Site**   | $E_\text{DFT}$ (eV) | $E_\text{ZPE}$ (eV) | $E_H$ (eV)  | -TS (eV)  | G (eV)   |
| ---------- | :-----------------: | :-----------------: | :--------:  | :------:  | :------: |
| H$_2$      |       &#8291;       |    &#8291;          |     &#8291; |    &#8291;|   &#8291;|
| $\ast$     |       &#8291;       |     0               |     0       |    0      |   &#8291;|
| H$^{\ast}$ |       &#8291;       |    &#8291;          |     &#8291; |    &#8291;|   &#8291;|

## Analysis

5. Plot a free energy diagram for the Volmer-Heyrovsky reaction at 0 V vs. RHE and at -0.2 V vs. RHE.

6. Identify the potential determining step (i.e. the step most 'uphill' in free energy) and the limiting potential step (i.e. the potential at which all steps become exergonic).

### Below we have provided you with some scripts you can use for inspiration

__As always, keep your scripts in separate folders and create directories before running a script that writes to that directory.__

In [ ]:
#%%writefile h2.py
from ase import Atoms
from ase.build import molecule
from ase.optimize import QuasiNewton
from ase.io import Trajectory
from ase.vibrations import Vibrations
from ase.thermochemistry import IdealGasThermo
from ase.parallel import paropen
from gpaw import GPAW, PW
from os.path import join

folder = 'h2'  # My folder name
folder = join(folder, '')  # ensure folder ends with a '/'

#System set-up
mol = molecule('H2')
mol.center(vacuum=5)    # Sets a supercell with 10 Å vacuum between neighbours
#mol.set_pbc(True)       # Sets periodic boundary conditions for the molecule; necessary for PW calculations
#print(mol.get_cell())  # Command to print the associated cell of the molecule
#print(mol.get_pbc())   # Command to print boundary conditions associated with the molecule

#Calculator set-up
calc = GPAW(h=.18,
            xc='RPBE',
            spinpol=False,
            txt=folder+'h2.txt')  # Output .txt file

mol.set_calculator(calc)

# Specify relaxation and trajectory
dyn = QuasiNewton(mol)            # Use the QuasiNewton approach for relaxation
traj = Trajectory(folder+'h2.traj', 'w', mol) # Specify trajectory object and location for relaxation
dyn.attach(traj)                  # Attach trajectory to optimizer
dyn.run(fmax=0.005)               # Run system relaxation to force threshold of 0.005 eV/Å

# Request energy output
en = mol.get_potential_energy()

# Conduct vibrational analysis
vib = Vibrations(mol,
                 name=folder+'vib',
                 nfree=2, delta=0.01) #Input system, 
vib.run()
vib_energies = vib.get_energies()
vib.summary()

for i in range(6):  # 3N vibrational modes
        vib.write_mode(i)
        
# Conduct thermochemistry analysis
thermo = IdealGasThermo(vib_energies=vib_energies, # Vibrational energies of the molecule
                        potentialenergy=en,         # Potential energy of molecule as calculated above
                        atoms=mol,                 # The actual atoms object
                        geometry='linear',         # H2 has a linear structure
                        symmetrynumber=2,
                        spin=0)


T = 298.15    # Temperature in K
P = 101325.   # 1 atm.
H = thermo.get_enthalpy(temperature=T)
G = thermo.get_gibbs_energy(temperature=T, pressure=P)
S = thermo.get_entropy(temperature=T, pressure=P)

# Write thermochemistry results to file
with paropen(folder+'h2_thermo.txt', 'w') as file:
    print('Harmonic limit thermochemistry', file=file)
    print('E(DFT):     {:.3f} eV'.format(en), file=file)
    print('H({:.2f} K): {:.3f} eV'.format(T, H), file=file)
    print('S({:.2f} K): {:.3f} eV'.format(T, S), file=file)
    print('G({:.2f} K): {:.3f} eV'.format(T, G), file=file)

In [ ]:
!qsub.py -t 1 -p 8 h2.py

In [ ]:
#%%writefile slab.py
# -----------------------------------------------------------------------------
# NOTE: PERFORM THIS CALCULATION ONLY IF YOU HAVE NOT ALREADY RELAXED A SURFACE
# -----------------------------------------------------------------------------
from ase import Atoms
from ase.build import fcc111
from ase.optimize import BFGS
from ase.constraints import FixAtoms
from gpaw import GPAW, PW, FermiDirac
from os.path import join

folder = 'slab'  # My folder name
folder = join(folder, '')  # ensure folder ends with a '/'

# Set up surface
slab = fcc111('Cu', (2,2,3), vacuum=7.5, a=3.679)  # Remember to update your lattice parameter
slab.set_pbc(True)

# Fix bottom layers to ease relaxation
mask=[atom.tag > 1 for atom in slab]
slab.set_constraint(FixAtoms(mask=mask))

# Define calculator object
calc = GPAW(mode=PW(400.),
            kpts=(4, 4, 1),
            xc='RPBE',
            occupations=FermiDirac(0.10),
            maxiter=600,
            spinpol=False,
            txt=folder+'slab.out', # Change to your preferred directory and filename. Ensure directory exists
            convergence={'energy': 0.0005,
                         'density': 1.0e-4,
                         'eigenstates': 4.0e-8,
                         'bands': 'occupied'})

slab.set_calculator(calc)

dyn = BFGS(slab, trajectory=folder+'slab.traj', logfile=folder+'slab.log')
dyn.run(fmax=0.05)

slab.get_potential_energy()

In [ ]:
!qsub.py -t 2 -p 8 slab.py

In [ ]:
#%%writefile slab_h_ads.py
# ---------------------------------------------------------------------------------------------
# NOTE: PERFORM THIS CALCULATION ONLY IF YOU HAVE NOT ALREADY ADSORBED A H ATOM TO YOUR SURFACE
# ---------------------------------------------------------------------------------------------
from ase import Atoms
from ase.build import add_adsorbate
from ase.io import read,write
from ase.parallel import paropen
from ase.optimize import BFGS
from gpaw import GPAW, PW, FermiDirac
from os.path import join

folder = 'slab_h_ads'  # My folder name
folder = join(folder, '')  # ensure folder ends with a '/'

# Read surface file
slab = read('slab/slab.traj')

add_adsorbate(slab, 'H', 1.3, position='fcc', offset=(0, 0))  # Add adsorbate to fcc position

# Define calculator object
calc = GPAW(mode=PW(400.),
            kpts=(4,4,1),
            xc='RPBE',
            occupations=FermiDirac(0.10),
            maxiter=600,
            spinpol=False,
            txt=folder+'slab_h_ads.out', # Change to your preferred directory and filename. Ensure directory exists
            convergence={'energy': 0.0005,
                         'density': 1.0e-4,
                         'eigenstates': 4.0e-8,
                         'bands': 'occupied'})

slab.set_calculator(calc)

dyn = BFGS(slab, trajectory=folder+'slab_h_ads.traj', logfile=folder+'slab_h_ads.log')
dyn.run(fmax=0.05)

en = slab.get_potential_energy()
with paropen(folder+'energy.dat', 'w') as file:
    print('E(DFT): {:.3f} eV'.format(en), file=file)

In [ ]:
!qsub.py -t 2 -p 8 slab_h_ads.py

In [ ]:
#%%writefile slab_h_vib_thermo.py
from ase.io import read
from ase.parallel import paropen
from ase.vibrations import Vibrations
from ase.thermochemistry import HarmonicThermo
from gpaw import GPAW, PW, FermiDirac
from os.path import join

slab = read('slab_h_ads/slab_h_ads.traj')

folder = 'slab_h_vib_thermo'  # My folder name
folder = join(folder, '')  # ensure folder ends with a '/'

### Define calculator object ###
calc = GPAW(mode=PW(400.),
            kpts=(4, 4, 1),
            xc='RPBE',
            occupations=FermiDirac(0.10),
            maxiter=600,
            spinpol=False,
            symmetry='off',  # Need to disable symmetry for vibrations
            txt=flsolder+'slab_h_vib_thermo.out', # Change to your preferred directory and filename. Ensure directory exists
            convergence={'energy': 0.0005,
                         'density': 1.0e-4,
                         'eigenstates': 4.0e-8,
                         'bands': 'occupied'})

slab.set_calculator(calc)

en = slab.get_potential_energy()

### Vibrational analysis ###
vibindices = (-1, ) # Atomic index for vibration; '-1' specifies last atom, i.e. H
vib = Vibrations(slab,
                 name=folder+'vib',
                 indices=vibindices,
                 nfree=2,
                 delta=0.01)
vib.run()
vib.summary()
for i in range(3):
    vib.write_mode(i)
    
vib_energies = vib.get_energies()

### Thermochemistry analysis ###
thermo = HarmonicThermo(vib_energies, E)
T = 298.15

U = thermo.get_internal_energy(temperature=T)
S = thermo.get_entropy(temperature=T)
F = thermo.get_helmholtz_energy(temperature=T)

with paropen(folder+'slab_h_thermo.dat', 'r') as file:
    print('Harmonic limit thermochemistry:', file=file)
    print('E(DFT:      {:.3f} eV'.format(en), file=file)
    print('U(298.15K): {:.3f} eV'.format(U), file=file)
    print('S(298.15K): {:.3f} eV'.format(S), file=file)
    print('F(298.15K): {:.3f} eV'.format(F), file=file)

In [ ]:
!qsub.py -t 2 -p 8 slab_h_vib_thermo.py